In [ ]:
import numpy as np
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit, Aer, execute
from qiskit.providers.aer.noise import NoiseModel, depolarizing_error

# Step 1: Build a simple noise model with depolarizing noise
def create_noise_model(error_prob):
    noise_model = NoiseModel()
    error_gate = depolarizing_error(error_prob, 1)
    noise_model.add_all_qubit_quantum_error(error_gate, ['u1', 'u2', 'u3'])
    return noise_model

# Step 2: Create different circuits to test noise models and choose the observable to measure
def create_test_circuit():
    qc = QuantumCircuit(1, 1)
    qc.h(0)
    qc.measure(0, 0)
    return qc

# Step 3: Apply the unitary folding method
def apply_unitary_folding(qc, repetitions):
    folded_qc = qc.copy()
    for _ in range(repetitions - 1):
        folded_qc += qc
    return folded_qc

# Step 4: Apply extrapolation method to get the zero-noise limit
def extrapolate_zero_noise(results, extrapolation_method):
    # Extract counts from results
    counts = results.get_counts()

    # Prepare data for extrapolation
    x = np.array(list(counts.keys())).astype(float)
    y = np.array(list(counts.values())).astype(float)

    # Define extrapolation function based on the chosen method
    if extrapolation_method == 'linear':
        def extrapolation_func(x, a, b):
            return a * x + b
    elif extrapolation_method == 'polynomial':
        def extrapolation_func(x, a, b, c):
            return a * x**2 + b * x + c
    elif extrapolation_method == 'exponential':
        def extrapolation_func(x, a, b):
            return a * np.exp(b * x)

    # Fit the data to the extrapolation function
    popt, _ = curve_fit(extrapolation_func, x, y)

    # Extrapolate to zero noise
    zero_noise_limit = extrapolation_func(0, *popt)

    return zero_noise_limit

# Step 5: Compare mitigated and unmitigated results
def compare_results(unmitigated_counts, mitigated_counts):
    print("Unmitigated Counts:", unmitigated_counts)
    print("Mitigated Counts:", mitigated_counts)

# Main function to run ZNE
def run_zne(error_prob, extrapolation_method):
    # Create noise model
    noise_model = create_noise_model(error_prob)

    # Create test circuit
    test_circuit = create_test_circuit()

    # Apply unitary folding
    folded_circuit = apply_unitary_folding(test_circuit, repetitions=5)

    # Simulate with noise
    simulator = Aer.get_backend('qasm_simulator')
    noisy_job = execute(folded_circuit, simulator, noise_model=noise_model, shots=1024)
    noisy_results = noisy_job.result()

    # Extrapolate zero noise limit
    zero_noise_limit = extrapolate_zero_noise(noisy_results, extrapolation_method)

    # Simulate without noise
    ideal_job = execute(folded_circuit, simulator, shots=1024)
    ideal_results = ideal_job.result()

    # Compare mitigated and unmitigated results
    unmitigated_counts = ideal_results.get_counts()
    mitigated_counts = noisy_results.get_counts()
    compare_results(unmitigated_counts, mitigated_counts)

    return zero_noise_limit

# Example usage
error_probability = 0.1
method = 'linear'
zero_noise = run_zne(error_probability, method)
print("Zero Noise Limit:", zero_noise)

# Plot the extrapolation curve
x_data = np.linspace(0, 1, 100)
plt.plot(x_data, extrapolation_func(x_data, *popt), label='Extrapolation Curve')
plt.xlabel('Noise Level')
plt.ylabel('Observables')
plt.title('Extrapolation to Zero Noise Limit')
plt.legend()
plt.show()
